## Always run thic CELL below
- To attach ESML controlplane to your project
- To point at `template-data` for the pipelinbe to know the schema of data
- To init the ESMLPieplinefactory

In [2]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
from baselayer_azure_ml_pipeline import ESMLPipelineFactory, esml_pipeline_types

p = ESMLProject()
p.inference_mode = True
p.active_model = 11 # 10=titanic , 11=Diabetes
p_factory = ESMLPipelineFactory(p)

scoring_date = '1000-01-01 10:35:01.243860' # 
p_factory.batch_pipeline_parameters[1].default_value = scoring_date # overrides ESMLProject.date_scoring_folder.
p_factory.describe()


Using lake_settings.json with ESML version 1.4 - Models array support including LABEL

 ---- Q: WHICH files are generated as templates, for you to EDIT? ---- 
A: These files & locations:
File to EDIT (step: IN_2_SILVER_1): ../../../2_A_aml_pipeline/4_inference/batch/M11/in2silver_ds01_diabetes.py
File to EDIT (step: IN_2_SILVER_2): ../../../2_A_aml_pipeline/4_inference/batch/M11/in2silver_ds02_other.py
File to EDIT (step: SILVER_MERGED_2_GOLD): ../../../2_A_aml_pipeline/4_inference/batch/M11/silver_merged_2_gold.py
File to EDIT (step: SCORING_GOLD): ../../../2_A_aml_pipeline/4_inference/batch/M11/scoring_gold.py
File to EDIT (step: TRAIN_SPLIT_AND_REGISTER): ../../../2_A_aml_pipeline/4_inference/batch/M11/train_split_and_register.py
File to EDIT (step: TRAIN_MANUAL): ../../../2_A_aml_pipeline/4_inference/batch/M11/train_manual.py
File to EDIT (step: TRAIN_AUTOML): ../../../2_A_aml_pipeline/4_inference/batch/M11/train_post_automl_step.py
File to EDIT a lot (reference in step-scripts Cus

# IN_2_GOLD_SCORING
- 1) Generate code files
- 2) Build pipepline, with the ESML oneliner, which also will upload the snapshot folder and code together with the Azure ML pipeline steps
- 3) Run the pipeline. Smoke testing, see that it works
- 4) IF it works, Publish the pipeline, or else, edit the code files or configuration, retry step 2 and 3.
- 5) Print the pipeline_id, that is essential to use from Azure Data factory

In [ ]:
# Genereate code files, to be edited later.
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT

In [3]:
## BUILD pipeline,
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD_SCORING) # Scoring
# ...which uses the BEST trained model, upload the genereated python scripts, uploads your custom code and ESML runtime, to Azure embedded in the pipeline, using Dockerized image. 

Did NOT overwrite script-files with template-files such as 'scoring_gold.py', since overwrite_if_exists=False
Using GEN2 as Datastore
ESML will auto-create a compute...
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Using a model specific cluster, per configuration in project specific settings, (the integer of 'model_number' is the base for the name)
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Found existing cluster p02-m11weu-dev for project and environment, using it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
image_build_compute = p02-m11weu-dev
Reusing existing compute...
create_gold_to_score_step: inference_mode=True


In [4]:
## RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

Execute_pipeline (scoring): Inference_mode: 1
-Scoring data, default value 1000-01-01 10:35:01.243860
Created step IN 2 SILVER - ds01_diabetes [50a68622][61116429-a69b-4767-8cd1-67d53c3bedd9], (This step will run and generate new outputs)
Created step IN 2 SILVER - ds02_other [d8161219][cbe5292f-1d3d-445b-a33d-79feba6a9a2e], (This step will run and generate new outputs)
Created step SILVER MERGED 2 GOLD [9ded6851][83b842f1-5866-4606-8224-53b4c64fb6ef], (This step will run and generate new outputs)
Created step SCORING GOLD [a1c16d92][62f97459-0b37-40df-9fbd-bd830bac0f93], (This step will run and generate new outputs)
Submitted PipelineRun b5e57980-770c-4d77-afb5-b98c92422539
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/b5e57980-770c-4d77-afb5-b98c92422539?wsid=/subscriptions/ca0a8c40-b06a-4e4e-8434-63c03a1dee34/resourcegroups/MSFT-WEU-EAP_PROJECT02_AI-DEV-RG/workspaces/msft-weu-DEV-eap-proj02_ai-amls&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
Pipeline submitted for ex

'Finished'

In [4]:
# PUBLISH
published_pipeline, endpoint = p_factory.publish_pipeline(batch_pipeline,"_1") # "_1" is optional    to create a NEW pipeline with 0 history, not ADD version to existing pipe & endpoint

pub_pipe.name 11_diabetes_model_reg_pipe_IN_2_GOLD_SCORING_EP_6
pub_pipe.id 01274209-95e8-4ffb-9b95-9bfb1c865b28
pub_pipe.name 11_diabetes_model_reg_pipe_IN_2_GOLD_SCORING_EP_1
pub_pipe.id cb6e1ef3-ddf2-483c-964e-bc6ba02c610f
pub_pipe.name 10_titanic_model_clas_pipe_IN_2_GOLD_SCORING_EP_1
pub_pipe.id 6c3c51e9-df26-4373-ae37-0a31a3616ec0


# Get info - to use  in Azure data factory: `published_pipeline.id` (if private Azure ML workspace)

In [5]:
print("2) Fetch scored data: Below needed for Azure Data factory PIPELINE activity (Pipeline OR Endpoint. Choose the latter") 
print ("- Endpoint ID")
print("Endpoint ID:  {}".format(endpoint.id))
print("Endpoint Name:  {}".format(endpoint.name))
print("Experiment name:  {}".format(p_factory.experiment_name))

print("In AZURE DATA FACTORY - This is the ID you need, if using PRIVATE LINK, private Azure ML workspace.")
print("-You need PIPELINE id, not pipeline ENDPOINT ID ( since cannot be chosen in Azure data factory if private Azure ML)")
published_pipeline.id

2) Fetch scored data: Below needed for Azure Data factory PIPELINE activity (Pipeline OR Endpoint. Choose the latter
- Endpoint ID
Endpoint ID:  6c3c51e9-df26-4373-ae37-0a31a3616ec0
Endpoint Name:  10_titanic_model_clas_pipe_IN_2_GOLD_SCORING_EP_1
Experiment name:  10_titanic_model_clas_pipe_IN_2_GOLD_SCORING
In AZURE DATA FACTORY - This is the ID you need, if using PRIVATE LINK, private Azure ML workspace.
-You need PIPELINE id, not pipeline ENDPOINT ID ( since cannot be chosen in Azure data factory if private Azure ML)


'0a3ffd65-f1e2-4705-8fd0-681e4d16fbe4'